<a href="https://colab.research.google.com/github/Only-Mike/DeepLearning-and-AI/blob/main/Assignment4_Question_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Load (some of) required libraries
!pip install -U transformers huggingface_hub datasets evaluate -qq
from datasets import load_dataset
import pandas as pd
import numpy as np



In [3]:
# Login to Hugging Face Hub to be able to save the model
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [9]:
#Dataset created from ChatGPT
import random

questions = [
    "What is ADHD?",
    "What are the symptoms of ADHD?",
    "How is ADHD diagnosed?",
    "What are the treatments for ADHD?",
    "Can ADHD be cured?",
    "What is the difference between ADD and ADHD?",
    "Is ADHD hereditary?",
    "What are some strategies for managing ADHD?",
    "Can ADHD affect adults?",
    "How does ADHD affect learning?",
    "How does ADHD affect sleep?",
    "What are some common misconceptions about ADHD?",
    "What are some resources for people with ADHD?",
    "How can ADHD affect relationships?",
    "What are some ways to talk to your doctor about ADHD?",
]

contexts = [
    "Attention-deficit/hyperactivity disorder (ADHD) is a neurodevelopmental disorder characterized by inattention, hyperactivity, and impulsivity.",
    "The symptoms of ADHD include difficulty paying attention, hyperactivity, and impulsivity.",
    "ADHD is typically diagnosed through a combination of assessments, including a medical evaluation, a clinical interview, and rating scales completed by parents, teachers, or caregivers.",
    "Treatments for ADHD may include medication, therapy, or a combination of both.",
    "ADHD cannot be cured, but symptoms can be managed with appropriate treatment.",
    "ADD (attention-deficit disorder) is an outdated term that was previously used to describe individuals with inattention but without hyperactivity or impulsivity. ADHD is now used to describe all presentations of the disorder.",
    "There is evidence to suggest that ADHD has a genetic component, but the exact cause is not yet fully understood.",
    "Some strategies for managing ADHD include medication, therapy, exercise, and organizational tools such as calendars and to-do lists.",
    "ADHD can affect adults, and symptoms may manifest differently than in children.",
    "ADHD can affect learning by making it difficult to focus and pay attention in class.",
    "ADHD can affect sleep by causing difficulties falling asleep or staying asleep.",
    "Some common misconceptions about ADHD include that it only affects children, that it is caused by poor parenting, and that it can be cured with willpower or discipline.",
    "Some resources for people with ADHD include support groups, therapy, and accommodations at school or work.",
    "ADHD can affect relationships by causing difficulties with communication, organization, and emotional regulation.",
    "Some ways to talk to your doctor about ADHD include bringing up specific symptoms, discussing any concerns or questions, and asking about available treatments and resources."
]

answers = [
    "ADHD is a neurodevelopmental disorder characterized by inattention, hyperactivity, and impulsivity.",
    "The symptoms of ADHD can include difficulty paying attention, hyperactivity, and impulsivity, among others.",
    "ADHD is typically diagnosed through a combination of assessments, including a medical evaluation, a clinical interview, and rating scales completed by parents, teachers, or caregivers.",
    "Treatments for ADHD may include medication, therapy, or a combination of both, and the best approach may vary depending on individual needs.",
    "ADHD cannot be cured, but symptoms can be managed with appropriate treatment.",
    "ADD (attention-deficit disorder) is an outdated term that was previously used to describe individuals with inattention but without hyperactivity or impulsivity. ADHD is now used to describe all presentations of the disorder.",
    "There is evidence to suggest that ADHD has a genetic component, but the exact cause is not yet fully understood.",
    "Some strategies for managing ADHD include medication, therapy, exercise, and organizational tools such as calendars and to-do lists.",
    "ADHD can affect adults, and symptoms may manifest differently than in children. It is important for adults to seek proper diagnosis and treatment if they suspect they have ADHD.",
    "ADHD can affect learning by making it difficult to focus and pay attention in class. However, with appropriate accommodations and support, individuals with ADHD can still succeed academically.",
    "ADHD can affect sleep by causing difficulties falling asleep or staying asleep. It is important to address sleep issues in individuals with ADHD, as poor sleep can exacerbate symptoms.",
    "Some common misconceptions about ADHD include that it only affects children, that it is caused by poor parenting, and that it can be cured with willpower or discipline. These misconceptions can lead to stigmatization and misunderstanding of the disorder.",
    "Some resources for people with ADHD include support groups, therapy, and accommodations at school or work. It is important for individuals with ADHD to seek out appropriate support and resources.",
    "ADHD can affect relationships by causing difficulties with communication, organization, and emotional regulation. It is important for individuals with ADHD and their loved ones to understand the impact of the disorder on relationships and to seek out appropriate support.",
    "Some ways to talk to your doctor about ADHD include bringing up specific symptoms, discussing any concerns or questions, and asking about available treatments and resources."
    ]

# Set the number of examples you want to create
num_examples = 50

# Create an empty list to store the examples
examples = []

# Loop through the number of examples and create a new row for each one
for i in range(num_examples):
    # Choose a random question and context from the lists
    question = random.choice(questions)
    context = random.choice(contexts)
    
    # Create a new row for the example and add it to the list of examples
    examples.append({
        "id": i,
        "title": "ADHD",
        "context": context,
        "question": question,
        "answers": {"text": ["Answer not available"], "answer_start": [0]}
    })

# Convert the list of examples to a pandas dataframe
df = pd.DataFrame(examples)

# Reorder the columns to match the desired format (not needed for now)
#df = 

In [10]:
df.head()

,id,title,context,question,answers
0,0,ADHD,ADHD can affect sleep by causing difficulties ...,What are some strategies for managing ADHD?,"{'text': ['Answer not available'], 'answer_sta..."
1,1,ADHD,Some resources for people with ADHD include su...,What are some resources for people with ADHD?,"{'text': ['Answer not available'], 'answer_sta..."
2,2,ADHD,ADHD can affect learning by making it difficul...,What is the difference between ADD and ADHD?,"{'text': ['Answer not available'], 'answer_sta..."
3,3,ADHD,ADHD is typically diagnosed through a combinat...,What is the difference between ADD and ADHD?,"{'text': ['Answer not available'], 'answer_sta..."
4,4,ADHD,Some strategies for managing ADHD include medi...,Can ADHD affect adults?,"{'text': ['Answer not available'], 'answer_sta..."


In [24]:
from datasets import Dataset
#Convert Pandas DataFrame to HF Dataframe
dataset = Dataset.from_pandas(df)

In [25]:
dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 50
})

In [26]:
# Split dataset into train/test with test size of 20%
dataset = dataset.train_test_split(test_size=0.2)

In [27]:
# Define the tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [28]:
# Define a preprocess function to format our input data
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [29]:
# Tokenize the dataset using the defined tokenizer and preprocess function
tokenized_squad = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [30]:
# Define data collator
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [31]:
# Define the specific model we'll use
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [35]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="ADHD_Test_qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    push_to_hub=True,
)
# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/content/ADHD_Test_qa_model is already a clone of https://huggingface.co/Only-Mike/ADHD_Test_qa_model. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimiz

Epoch,Training Loss,Validation Loss
1,No log,0.086006
2,No log,0.046972
3,No log,0.027513
4,No log,0.014022
5,No log,0.008210
6,No log,0.005937
7,No log,0.004665
8,No log,0.003910
9,No log,0.003536
10,No log,0.003221


***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
***** Running Evaluation *****
  N

TrainOutput(global_step=45, training_loss=0.027264573838975695, metrics={'train_runtime': 24.8228, 'train_samples_per_second': 24.171, 'train_steps_per_second': 1.813, 'total_flos': 58793895014400.0, 'train_loss': 0.027264573838975695, 'epoch': 15.0})

In [38]:
trainer.push_to_hub()

Saving model checkpoint to ADHD_Test_qa_model
Configuration saved in ADHD_Test_qa_model/config.json
Model weights saved in ADHD_Test_qa_model/pytorch_model.bin
tokenizer config file saved in ADHD_Test_qa_model/tokenizer_config.json
Special tokens file saved in ADHD_Test_qa_model/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 32.0k/253M [00:00<?, ?B/s]

Upload file runs/Mar10_09-03-35_9806c7c2ac86/events.out.tfevents.1678439018.9806c7c2ac86.580.2: 100%|#########…

Upload file runs/Mar10_09-05-09_9806c7c2ac86/events.out.tfevents.1678439112.9806c7c2ac86.580.6: 100%|#########…

Upload file runs/Mar10_09-02-19_9806c7c2ac86/1678438949.8770425/events.out.tfevents.1678438949.9806c7c2ac86.58…

...

Upload file runs/Mar10_09-04-15_9806c7c2ac86/1678439058.706274/events.out.tfevents.1678439058.9806c7c2ac86.580…

Upload file runs/Mar10_09-02-19_9806c7c2ac86/events.out.tfevents.1678438949.9806c7c2ac86.580.0: 100%|#########…

Upload file training_args.bin: 100%|##########| 3.43k/3.43k [00:00<?, ?B/s]

remote: Scanning LFS files of refs/heads/main for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/Only-Mike/ADHD_Test_qa_model
   6643851..6f5d3f2  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Only-Mike/ADHD_Test_qa_model
   6643851..6f5d3f2  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}
To https://huggingface.co/Only-Mike/ADHD_Test_qa_model
   6f5d3f2..512f44c  main -> main

   6f5d3f2..512f44c  main -> main



'https://huggingface.co/Only-Mike/ADHD_Test_qa_model/commit/6f5d3f2aad8096a205f86917be35011da72e9152'

In [39]:
question = "What is ADHD?"
context = "Attention-deficit/hyperactivity disorder (ADHD) is a neurodevelopmental disorder characterized by inattention, hyperactivity, and impulsivity."

In [40]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="ADHD_Test_qa_model")
question_answerer(question=question, context=context)

loading configuration file ADHD_Test_qa_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "ADHD_Test_qa_model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading configuration file ADHD_Test_qa_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "ADHD_Test_qa_model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings

{'score': 0.9964203238487244,
 'start': 0,
 'end': 31,
 'answer': 'Attention-deficit/hyperactivity'}